# Point of this file:
compare output of entropy transfer implemented in GS2 and output from python script calculation with dist fn print out at last timestep <br>

**Note:** script for this test now in crossTest.py

In [1]:
import netCDF4 as nc
import numpy.ma as ma
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### load imlemented calculation result

In [2]:
data = nc.Dataset("nlinear1/test4/c1.out.nc")

In [3]:
data["entropy_transfer_4D"]

<class 'netCDF4._netCDF4.Variable'>
float64 entropy_transfer_4D(t, ky, ky, kx, kx)
    description: entropy transfer(kys,kyt,kxs,kxt) with ky extended, i.e. 2*nky-1 of what is stated in dim_string
    units: no units
unlimited dimensions: t
current shape = (7, 6, 6, 11, 11)
filling on, default _FillValue of 9.969209968386869e+36 used

In [4]:
S_transfer = np.array(data["entropy_transfer_4D"][:])

### load result of post run python script

In [5]:
data = nc.Dataset("nlinear1/test4/c1.out.nc") #test4 folder is baseline that works
S_transfer = np.array(data["entropy_transfer_4D"][:])

S_transfer_python = xr.open_dataset("entropy_transfer.nc")
S_transfer_python_sum = np.sum(S_transfer_python["entropy_transfer"],axis=(0,1,2,3)) #sum over theta,energy,lambda,sign

#take out result of ky >= 0, negative kys result is symmetric to positive ones
ky = S_transfer_python["kys"].values
nky = len(ky)
iky0 = np.argmin(np.abs(ky))


S_transfer_python_sum = S_transfer_python_sum[iky0:,iky0:,:,:]

A = S_transfer[-1,:,:,:,:]
B = S_transfer_python_sum

In [6]:
S_transfer_python = xr.load_dataset("entropy_transfer.nc")

In [7]:
S_transfer_python

<xarray.Dataset>
Dimensions:           (theta: 13, energy: 6, lambda: 10, sign: 2, kys: 11,
                       kyt: 11, kxs: 11, kxt: 11)
Coordinates:
  * theta             (theta) float64 -3.142 -2.618 -2.094 ... 2.094 2.618 3.142
  * energy            (energy) float64 0.07939 1.562 4.921 6.666 8.544 12.54
  * lambda            (lambda) float64 0.1388 0.534 0.783 ... 1.09 1.156 1.18
  * sign              (sign) int64 0 1
  * kys               (kys) float64 -1.0 -0.8 -0.6 -0.4 -0.2 ... 0.4 0.6 0.8 1.0
  * kyt               (kyt) float64 -1.0 -0.8 -0.6 -0.4 -0.2 ... 0.4 0.6 0.8 1.0
  * kxs               (kxs) float64 -1.005 -0.8042 -0.6032 ... 0.8042 1.005
  * kxt               (kxt) float64 -1.005 -0.8042 -0.6032 ... 0.8042 1.005
Data variables:
    entropy_transfer  (theta, energy, lambda, sign, kys, kyt, kxs, kxt) float64 ...

In [8]:
S_transfer_python["entropy_transfer"].shape

(13, 6, 10, 2, 11, 11, 11, 11)

In [9]:
S_transfer_python_sum = np.sum(S_transfer_python["entropy_transfer"],axis=(0,1,2,3))

#take out result of ky >= 0, negative kys result is symmetric to positive ones
ky = S_transfer_python["kys"].values
nky = len(ky)
iky0 = np.argmin(np.abs(ky))

S_transfer_python_sum = S_transfer_python_sum[iky0:,iky0:,:,:]

In [10]:
S_transfer_python_sum.shape, S_transfer.shape

((6, 6, 11, 11), (7, 6, 6, 11, 11))

### compare random values for impression

In [11]:
#S_transfer_python_sum[2,1,:,:]

In [12]:
#S_transfer[-1,2,1,:,:]

### compare both fully up to certain sensible certainty

In [13]:
A = np.round(S_transfer[-1,:,:,:,:],8)
B = np.round(S_transfer_python_sum.values,8)

(A==B).all()

True

-> implementation works. Comparison script: crossTest.py

### test symmetries

In [14]:
S_transfer_python_sum = np.sum(S_transfer_python["entropy_transfer"],axis=(0,1,2,3))

In [15]:
S_transfer_python_sum.shape

(11, 11, 11, 11)

In [16]:
kx = S_transfer_python["kxs"].values
ky = S_transfer_python["kys"].values
nkx = len(kx)
nky = len(ky)

In [17]:
import sys
sys.path.append('/users/tms535/Developer/zonal-transfer-functions/entropy_transfer/local_Mac_testing/indexing-sym/')
import helpers

In [20]:
helpers.symmetry_test1(kx,ky,S_transfer_python_sum.values,rel_tol=0.0)

  0%|                                                                                                                                                         | 0/11 [00:00<?, ?it/s]

error -0.0002505441199288141 -0.00022197618657027096 -2.8567933358543163e-05
error -0.0006805058283845811 -0.0005517982647719011 -0.00012870756361268
error -0.0007697786323221668 -0.0005874482766137931 -0.00018233035570837364
error -0.0005450760540988715 -0.0004014857224678499 -0.00014359033163102155
error -0.0003527729106645687 -0.00025682887483681363 -9.594403582775508e-05
error -0.00022197618657027096 -0.0002505441199288141 2.8567933358543163e-05
error -0.0002293290443507243 -0.00020988691829841697 -1.944212605230734e-05
error -0.0005247272260954304 -0.0004519758284847492 -7.275139761068116e-05
error -0.0005288345020671285 -0.00043965357839340243 -8.918092367372606e-05
error -0.0004323911502193688 -0.00035530657631902284 -7.708457390034597e-05
error -0.00046817007914199034 -0.0003892194056683756 -7.895067347361476e-05


False

In [21]:
helpers.symmetry_test2(kx,ky,S_transfer_python_sum.values)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 49.95it/s]


True

In [22]:
helpers.symmetry_test3(kx,ky,S_transfer_python_sum.values)

  0%|                                                                                                                                                         | 0/11 [00:00<?, ?it/s]

error 0.00012452528388817713 -1.0902432984104503e-05 0.00013542771687228165
error 0.0004684774450956484 0.00011496768524228045 0.00035350975985336795
error 0.0004444168964039152 0.0001549722241543326 0.0002894446722495826
error 5.704443555531189e-05 -4.390769700846089e-06 6.143520525615797e-05
error -0.0003210305040362371 -0.0002336485662317654 -8.73819378044717e-05
error -0.00020260301027512703 -0.00023823856355679958 3.5635553281672546e-05
error 7.063295798339777e-05 -1.935547043717952e-05 8.998842842057729e-05
error 0.00024506625065912513 5.2234528239367926e-05 0.0001928317224197572
error 0.00013785926896914714 2.046053243103577e-05 0.00011739873653811136
error -0.0001584779256323399 -0.0001514138232578941 -7.064102374445793e-06
error -0.0003912883234021455 -0.0003258498759765087 -6.543844742563682e-05


False